In [25]:
import pandas as pd
import numpy as np
#import tensorflow as tf
from glob import glob
import shutil
import sys 
import os
from tqdm import tqdm

import torch
import yaml

In [9]:
#df = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
#df = df[df['class_id'] != 14]
df = pd.read_csv('../input/vinbigdata-1024-image-dataset/vinbigdata/train.csv')
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072


In [10]:
path = '../input/vinbigdata-1024-image-dataset/vinbigdata/train/'
df['image_path'] = df.image_id.apply(lambda x: path + x + '.png')

In [11]:
# train only 14 class
df = df[df.class_id != 14].reset_index(drop=True)
df['x_min'] = df.apply(lambda x: (x.x_min)/x.width, axis = 1)
df['x_max'] = df.apply(lambda x: (x.x_max)/x.width, axis = 1)
df['y_min'] = df.apply(lambda x: (x.y_min)/x.height, axis = 1)
df['y_max'] = df.apply(lambda x: (x.y_max)/x.height, axis = 1)
df['x'] = df.apply(lambda x: (x.x_max + x.x_min)/2, axis = 1)
df['y'] = df.apply(lambda x: (x.y_max + x.y_min)/2, axis = 1)
df['w'] = df.apply(lambda x: (x.x_max - x.x_min), axis = 1)
df['h'] = df.apply(lambda x: (x.y_max - x.y_min), axis = 1)
df['area'] = df.w*df.h
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path,x,y,w,h,area
0,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,0.332212,0.588613,0.794712,0.783818,2080,2336,../input/vinbigdata-1024-image-dataset/vinbigd...,0.563462,0.686216,0.462500,0.195205,0.090283
1,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,0.548611,0.257986,0.699219,0.353819,2304,2880,../input/vinbigdata-1024-image-dataset/vinbigd...,0.623915,0.305903,0.150608,0.095833,0.014433
2,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,0.246850,0.116211,0.372835,0.140951,2540,3072,../input/vinbigdata-1024-image-dataset/vinbigd...,0.309843,0.128581,0.125984,0.024740,0.003117
3,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,0.589497,0.095890,0.957549,0.848924,2285,2555,../input/vinbigdata-1024-image-dataset/vinbigd...,0.773523,0.472407,0.368053,0.753033,0.277156
4,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,0.216900,0.701461,0.262850,0.740829,2568,3353,../input/vinbigdata-1024-image-dataset/vinbigd...,0.239875,0.721145,0.045950,0.039368,0.001809


In [12]:
train_feats = ['x_min', 'y_min', 'x_max', 'y_max', 'x', 'y', 'w', 'h', 'area']
X_data = df[train_feats]
y_data = df['class_id']
#X_data.shape
#y_data.shape

In [15]:
unique = df.drop_duplicates('class_name')
class_name = unique['class_name'].tolist()
class_id = unique['class_id'].tolist()
class_name, class_id

(['Cardiomegaly',
  'Aortic enlargement',
  'Pleural thickening',
  'ILD',
  'Nodule/Mass',
  'Pulmonary fibrosis',
  'Lung Opacity',
  'Atelectasis',
  'Other lesion',
  'Infiltration',
  'Pleural effusion',
  'Calcification',
  'Consolidation',
  'Pneumothorax'],
 [3, 0, 11, 5, 8, 13, 7, 1, 9, 6, 10, 2, 4, 12])

In [22]:
val_size = int(df.shape[0]*0.8)
train_files = df.image_path[:val_size].tolist()
val_files = df.image_path[val_size:].tolist()
len(train_files), len(val_files)

(28876, 7220)

In [24]:
os.makedirs('/kaggle/working/vinbigdata/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/val', exist_ok = True)
label_dir = '/kaggle/input/vinbigdata-yolo-labels-dataset/labels'
for file in tqdm(train_files):
    shutil.copy(file, '/kaggle/working/vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/train')
    
for file in tqdm(val_files):
    shutil.copy(file, '/kaggle/working/vinbigdata/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/val')

100%|██████████| 7220/7220 [00:46<00:00, 155.18it/s]


In [28]:
# yolo training part
cwd = '/kaggle/working/'

with open(os.path.join(cwd , 'train.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata/images/train/*'):
        f.write(path+'\n')
            
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata/images/val/*'):
        f.write(path+'\n')

data = dict(train = os.path.join(cwd , 'train.txt') ,
            val = os.path.join(cwd , 'val.txt' ),
            nc = 14,
            names = class_name)

with open(os.path.join(cwd , 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join(cwd , 'vinbigdata.yaml'), 'r')

In [34]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')
print('Using device : %s' % (torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using device : CPU


In [ ]:
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 30 --data /kaggle/working/vinbigdata.yaml --weights yolov5x.pt --cache

2021-07-23 10:01:31.886025: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-07-23 10:01:31.886086: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-07-23 10:01:37.352435: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-07-23 10:01:37.352517: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
Scanning '/kaggle/working/vinb

1
